In [19]:
import pandas as pd
import numpy as np
import random
from IPython.display import clear_output
import time

from transformers import BertTokenizer, TFBertModel

In [15]:
df1 = pd.read_csv('All_US_tweets.csv')

In [32]:
df1['TweetText'] = df1['TweetText'].astype(str)
df1 = df1[df1['TweetText'].str.split(" ").str.len() <= 29]
df1["TweetText"].str.split(" ").str.len().describe(percentiles=[0.01, 0.5, 0.99])

count    371389.000000
mean         11.708861
std           6.916713
min           1.000000
1%            2.000000
50%          10.000000
99%          28.000000
max          29.000000
Name: TweetText, dtype: float64

In [45]:
df2 = df1.head(500).copy()

In [20]:
b_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [34]:
def tokenize_tweet(tweet):
    return b_tokenizer(tweet, padding='max_length', truncation=True, return_tensors="tf")

def get_bert_embeddings(tokenized_tweet):
    outputs = bert_model(tokenized_tweet)
    return outputs.last_hidden_state[:, 0, :]


In [46]:
df2['TweetText'] = df2['TweetText'].apply(tokenize_tweet)

In [47]:
start = time.time()
df2['embeddings'] = df2['TweetText'].apply(get_bert_embeddings)

end = time.time()

In [48]:
total_time = end - start
hours = total_time // 3600
mins = total_time % 3600 // 60
sec = total_time % 3600 % 60

print("Execution took: " + str(hours) + ' hrs, ' + str(mins) + ' min, ' + str(np.round(sec,2)) + 'secs')

Execution took: 0.0 hrs, 12.0 min, 52.88secs
